## Se descarga el codigo de Kaggle, ya que este viene de multiples fuentes

In [8]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F63%2F589%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240316%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240316T222831Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db88a09aef4289d7b06fc649c0d4692c08bed79c3ef80d324eab9e484ee302e9bfb733ecb9a0a48a9e358931eb72141ba0bbc936bee3feac070c9b06e2d08193dc1761f951d70467a749e603efdf8ccb26275c40f9c0cba155223ad1a7a375087f94f51be1ac18f8c7edd5db80011c883b722e8197aca31ff08c4ec180fc6b16303bed9415d039ca773eee4cfbf0774ffe0f9d9d0a4a9a98da3f7f82bdb9fdb5786f30c4f5715d39474003ffd092165f5e44478bd92bdf34f4dc2bd35c3f82f41ee90159c038ce53b5a2b65dcf1d5ab8150a8a4cfd69d3be72baef1274380b4ac8df7ea0a2ced66dacbae3aef9ea064460a6063f4cc142d8fdaa5de4331509f41'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 34297253 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


## Luego se importan varias bibliotecas a usar en el proyecto para el análisis de datos

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

path = "../input/"  #Insert path here
database = path + 'database.sqlite'

## Creamos la conexión con la  base de datos

In [10]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


# iii. Para el dataset de fútbol responder:

## 6. ¿Cuántos jugadores hay en la base de datos?

In [26]:
player = pd.read_sql("SELECT * FROM Player", conn)

jugadores = player['player_api_id'].nunique()  # Contar los jugadores únicos por su ID
print("En la base de datos hay", jugadores, "jugadores")

En la base de datos hay 11060 jugadores


## 7. ¿Cuáles son los 10 primeros jugadores en la tabla player?

In [31]:
player = pd.read_sql("SELECT * FROM Player", conn)

primeros_10_nombres = player['player_name'].head(10)
print("Los 10 primeros jugadores son:")
print("\n")
print(primeros_10_nombres)

Los 10 primeros jugadores son:


0    Aaron Appindangoye
1       Aaron Cresswell
2           Aaron Doran
3         Aaron Galindo
4          Aaron Hughes
5            Aaron Hunt
6            Aaron Kuhl
7          Aaron Lennon
8          Aaron Lennox
9         Aaron Meijers
Name: player_name, dtype: object


##8. ¿Cuántos países hay? ¿Cuántos jugadores hay en cada país?

In [68]:
country = pd.read_sql("SELECT * FROM country", conn)

paises = country['name'].nunique()
print("\n")
print("El número de países en la base de datos es de", paises)

print("\nNo es posible obtener número de jugadores por país")



El número de países en la base de datos es de 11

No es posible obtener número de jugadores por país


##9. ¿Cuántas ligas hay?


In [76]:
league = pd.read_sql("SELECT * FROM league", conn)

ligas = league['name'].nunique()
print("\n")
print("Hay", ligas, "ligas")
print("\n")
print(league['name'])



Hay 11 ligas


0       Belgium Jupiler League
1       England Premier League
2               France Ligue 1
3        Germany 1. Bundesliga
4                Italy Serie A
5       Netherlands Eredivisie
6           Poland Ekstraklasa
7     Portugal Liga ZON Sagres
8      Scotland Premier League
9              Spain LIGA BBVA
10    Switzerland Super League
Name: name, dtype: object


##10.  ¿Cuántas ligas hay por país?

In [145]:
# Unir las tablas 'League' y 'Country' usando 'country_id' y 'id' respectivamente
ligas_pais = league.merge(country, left_on='country_id', right_on='id')

print("El número de ligas por país es:")

# Contar el número de ligas por país
ligas_por_pais = ligas_pais['name_y'].value_counts()

print("\n")
print(ligas_por_pais)

El número de ligas por país es:


Belgium        1
England        1
France         1
Germany        1
Italy          1
Netherlands    1
Poland         1
Portugal       1
Scotland       1
Spain          1
Switzerland    1
Name: name_y, dtype: int64
